# Streamed processing with Loki workflow

In [ ]:
import scipp as sc
from ess import sans
from ess import loki
import ess.loki.data  # noqa: F401
from ess.sans.types import *

## Create and configure the workflow

We begin by creating the Loki workflow object (this is a [sciline.Pipeline](https://scipp.github.io/sciline/generated/classes/sciline.Pipeline.html) which can be consulted for advanced usage).
The files we use here come from a Loki detector test at Larmor, so we use the corresponding workflow:

In [ ]:
workflow = loki.LokiAtLarmorWorkflow()

We configure the workflow be defining the series of masks filenames and bank names to reduce.
In this case there is just a single bank:

In [ ]:
workflow = sans.with_pixel_mask_filenames(
    workflow, masks=loki.data.loki_tutorial_mask_filenames()
)
workflow[NeXusDetectorName] = 'larmor_detector'

The workflow can be visualized as a graph.
For readability we show only sub-workflow for computing `IofQ[Sample]`.
The workflow can actually compute the full `BackgroundSubtractedIofQ`, which applies and equivalent workflow to the background run, before a subtraction step:

In [ ]:
workflow.visualize(IofQ[SampleRun], compact=True, graph_attr={'rankdir': 'LR'})
wf = workflow.copy()
wf[TransmissionFraction[SampleRun]] = None
wf.visualize(IofQ[SampleRun], compact=True, graph_attr={'rankdir': 'LR'})

Note the red boxes which indicate missing input parameters.
We can set these missing parameters, as well as parameters where we do not want to use the defaults:

In [ ]:
# Wavelength binning parameters
wavelength_min = sc.scalar(1.0, unit='angstrom')
wavelength_max = sc.scalar(13.0, unit='angstrom')
n_wavelength_bins = 50
n_wavelength_bands = 50

workflow[WavelengthBins] = sc.linspace(
    'wavelength', wavelength_min, wavelength_max, n_wavelength_bins + 1
)
workflow[WavelengthBands] = None


workflow[CorrectForGravity] = True
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
workflow[ReturnEvents] = False

workflow[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.3, num=101, unit='1/angstrom')
workflow[DirectBeam] = None

## Configuring data to load

We have not configured which files we want to load.
In this tutorial, we use helpers to fetch the tutorial data which return the filenames of the cached files.
In a real use case, you would set these parameters manually:

In [ ]:
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_sample_run_60339()
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_sample_run_60250()
workflow[Filename[BackgroundRun]] = loki.data.loki_tutorial_background_run_60393()
workflow[Filename[TransmissionRun[SampleRun]]] = (
    loki.data.loki_tutorial_sample_transmission_run()
)
workflow[Filename[TransmissionRun[BackgroundRun]]] = loki.data.loki_tutorial_run_60392()
workflow[Filename[EmptyBeamRun]] = loki.data.loki_tutorial_run_60392()

## Finding the beam center

Looking carefully at the workflow above,
one will notice that there is a missing parameter from the workflow: the red box that contains the `BeamCenter` type.
Before we can proceed with computing the direct beam function,
we therefore have to first determine the center of the beam.

There are more details on how this is done in the [beam center finder notebook](../common/beam-center-finder.ipynb),
but for now we simply reuse the workflow (by making a copy),
and inserting the provider that will compute the beam center.

For now, we compute the beam center only for the rear detector (named 'larmor_detector') but apply it to all banks (currently there is only one bank).
The beam center may need to be computed or applied differently to each bank, see [scipp/esssans#28](https://github.com/scipp/esssans/issues/28).
We use a center-of-mass approach to find the beam center:

In [ ]:
center = sans.beam_center_from_center_of_mass(workflow)
center

and set that value in our workflow

In [ ]:
workflow[BeamCenter] = center

## Fake streaming workflow

In [ ]:
from ess.reduce.nexus.generic_types import NeXusDetectorEventData, NeXusMonitorEventData

det_events = workflow.compute(NeXusDetectorEventData[SampleRun])
mon_events = workflow.compute(NeXusMonitorEventData[SampleRun, Incident])
det_events

In [ ]:
mon_events

In [ ]:
%matplotlib widget
dummy = sc.DataArray(
    sc.zeros(dims=('Q',), shape=(100,), with_variances=True),
    coords={'Q': sc.linspace('Q', 0.0, 1.0, 101, unit='1/angstrom')},
)
fig = dummy.plot(norm='log', scale={'Q': 'log'}, vmin=0.1, vmax=5)
fig.canvas.ylabel = '$I(Q)$ ' + fig.canvas.ylabel
artist = next(iter(fig.artists))
display(fig)

In [ ]:
from ess.reduce import streaming

streaming_wf = streaming.StreamProcessor(
    base_workflow=workflow,
    dynamic_keys=(
        NeXusMonitorEventData[SampleRun, Incident],
        NeXusDetectorEventData[SampleRun],
    ),
    accumulators={
        ReducedQ[SampleRun, Numerator]: streaming.RollingAccumulator(window=5),
        ReducedQ[SampleRun, Denominator]: streaming.RollingAccumulator(window=5),
    },
    target_keys=(IofQ[SampleRun],),
)


det_stride = 60
mon_stride = 1
for i in range(100):
    det_chunk = det_events[det_stride * i : det_stride * (i + 1)].copy()
    mon_chunk = mon_events[mon_stride * i : mon_stride * (i + 1)].copy()
    if 20 < i < 30:
        det_chunk *= 0.0
    results = streaming_wf.add_chunk(
        {
            NeXusDetectorEventData[SampleRun]: det_chunk,
            NeXusMonitorEventData[SampleRun, Incident]: mon_chunk,
        }
    )
    fig.update({artist: results[IofQ[SampleRun]]})
    fig.fig.canvas.draw()
    fig.fig.canvas.flush_events()

## Select range with slider

In [ ]:
from ess.reduce import streaming
from ess.reduce.nexus.generic_types import PulseSelection
from ess.reduce.nexus.types import PreopenNeXusFile
from ess.reduce.nexus import generic_types as gt


def hack_monitor_events_by_name(
    filename: gt.NeXusFileSpec[gt.RunType],
    name: gt.NeXusMonitorName[gt.MonitorType],
    selection: PulseSelection[gt.RunType],
) -> gt.NeXusMonitorEventLocationSpec[gt.RunType, gt.MonitorType]:
    scaled = selection.value
    if isinstance(scaled, slice) and scaled.start is not None:
        fudge = 60  # file has 60x fewer monitor pulses than det pulses
        fudge = 5
        start = int(scaled.start // fudge) % 100
        stop = int(scaled.stop // fudge) % 100
        scaled = slice(start, stop)
    return gt.NeXusMonitorEventLocationSpec[gt.RunType, gt.MonitorType](
        filename=filename.value,
        component_name=name,
        selection={'event_time_zero': scaled},
    )


base_workflow = workflow.copy()
base_workflow[PreopenNeXusFile] = True
base_workflow.insert(hack_monitor_events_by_name)


streaming_wf = streaming.StreamProcessor(
    base_workflow=base_workflow,
    dynamic_keys=(PulseSelection[SampleRun],),
    accumulators=(),
    # target_keys=(WavelengthMonitor[SampleRun, Incident],),
    target_keys=(IofQ[SampleRun],),
)

In [ ]:
%matplotlib widget
import plopp
import ipywidgets as widgets
from time import time

pulse_sel_slider = widgets.IntSlider(
    value=3,
    min=0,
    max=6000,
    step=100,
    description='First pulse:',
    continuous_update=True,
)
pulse_width_slider = widgets.IntSlider(
    value=60,
    # min=60,  # cannot go lower since this is monitor granularity
    min=5,
    max=200,
    step=1,
    description='Pulse count:',
    continuous_update=True,
)
start_node = plopp.widget_node(pulse_sel_slider)
width_node = plopp.widget_node(pulse_width_slider)

wav = sc.linspace('wavelength', 1.0, 13.0, 100, unit='angstrom')


@plopp.node
def update(start, width):
    end = start + width
    sel = PulseSelection(slice(start, end))
    start = time()
    results = streaming_wf.add_chunk({PulseSelection[SampleRun]: sel})
    # print(f'Processed in {time() - start:.2f} s')
    return next(iter(results.values()))
    mon = results[WavelengthMonitor[SampleRun, Incident]]
    return mon.hist(wavelength=wav)


update_node = update(start=start_node, width=width_node)

f = plopp.plot(update_node)
f.bottom_bar.add(pulse_sel_slider)
f.bottom_bar.add(pulse_width_slider)
f

### Debugging files

In [ ]:
%matplotlib widget
wav = sc.linspace('wavelength', 1.0, 13.0, 100, unit='angstrom')
dummy = sc.DataArray(
    sc.zeros(dims=('wavelength',), shape=(100,), with_variances=True, unit='counts'),
    coords={'wavelength': wav},
)
fig = dummy.plot()
fig.canvas.ylabel = '$Incident monitor$ ' + fig.canvas.ylabel
artist = next(iter(fig.artists))
display(fig)

In [ ]:
import ipywidgets as widgets

pulse_sel_slider = widgets.IntSlider(
    value=0, min=0, max=100, step=100, description='Pulse:', continuous_update=False
)


# Note updates glitch
def update(change):
    start = change.new
    end = start + 10
    sel = PulseSelection(slice(start, end))
    results = streaming_wf.add_chunk({PulseSelection[SampleRun]: sel})
    mon = results[WavelengthMonitor[SampleRun, Incident]]
    # print(f'Loaded {mon.bins.size().sum().value} events')
    fig.update({artist: mon.hist(wavelength=wav)})
    fig.fig.canvas.draw()
    fig.fig.canvas.flush_events()


pulse_sel_slider.observe(update, names='value')
display(pulse_sel_slider)

In [ ]:
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_sample_run_60339()
workflow[Filename[SampleRun]] = loki.data.loki_tutorial_sample_run_60250()
mon = workflow.compute(MonitorData[SampleRun, Incident]).hist()
mon.plot()

In [ ]:
%matplotlib widget
from ess.reduce.nexus.generic_types import NeXusDetectorEventData

det = workflow.compute(NeXusDetectorEventData[SampleRun]).hist()
det.plot()
det.bin(event_time_zero=mon.coords['event_time_zero'][55:]).hist().plot()

In [ ]:
t_mon = workflow.compute(MonitorData[SampleRun, Incident]).coords['event_time_zero']
t_det = workflow.compute(NeXusDetectorEventData[SampleRun]).coords['event_time_zero']

In [ ]:
t_mon.plot()

In [ ]:
t_det.plot()